# Convert Moldflow meshes to GMSH with custom meshio version

See https://github.com/meyer-nils/meshio

In [11]:
import glob
import meshio
import numpy as np

files = sorted(glob.glob("data/*/*.pat", recursive="True"))

for file in files:
    # Read files
    filltime = file.replace(".pat", "_filltime.xml")
    orientation = file.replace(".pat", "_orientation.xml")
    mesh = meshio.moldflow.read(
        file, scale=1000.0, xml_filenames=[filltime, orientation]
    )

    # Make points 2D 
    mesh.points[:, 2] = 0.0

    # Extend symmetric tensors to 9 elements for MSH format
    for label in mesh.cell_data:
        for i, cell_data in enumerate(mesh.cell_data[label]):
            if cell_data.ndim > 1:
                mesh.cell_data[label][i] = np.hstack([cell_data, cell_data[:, 3:6]])

    # Export meshes
    meshio.vtu.write(file.replace(".pat", ".vtu"), mesh, binary=True)
    meshio.gmsh.write(file.replace(".pat", ".msh"), mesh, binary=False)
